# Integración del Agente

## Objetivo
Este notebook explora cómo nuestro agente se integra dentro del ecosistema más amplio, incluyendo la interacción con otros agentes, subagentes, sistemas externos y el framework de orquestación. Estableceremos los mecanismos de comunicación y coordinación necesarios para el funcionamiento cohesivo del sistema.

En este notebook:
- Implementaremos protocolos de comunicación entre agentes
- Desarrollaremos mecanismos para la gestión de subagentes
- Configuraremos la integración con sistemas y APIs externos
- Estableceremos la integración con el sistema de orquestación central

In [1]:
# %pip install langchain_community

In [2]:
from os import chdir

chdir("../")

In [3]:
from core.agent_state import AgentState
from typing import TypedDict, List, Optional
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI
import json
import os
import requests
from langgraph.graph import StateGraph, END, START
from nodes.order_tasks import order_tasks
from nodes.classify_query import classify_tasks
from nodes.error_handler import error_handler
from nodes.aggregator_tasks import aggregator

from agents.currency_agent import get_exchange_rate
from agents.news_agent import get_news
from agents.weather_agent import get_weather


In [4]:

def check_completion(state: AgentState):
    # Toma todas las llaves actuales de tareas completadas
    completed_tasks = list(state.get("task_completed", {}).keys())

    print(f"Tareas a verificar: {completed_tasks}")
    print(state)

    # Verifica que todas las tareas listadas estén marcadas como completadas (valor True)
    all_completed = all(state["task_completed"].get(task, False) for task in completed_tasks)

    print(f"Verificando completitud: {all_completed}")
    print(f"Estado actual: {state['task_completed']}")

    # Si todas están completas, marcamos como listo para agregar
    if all_completed:
        state["ready_to_aggregate"] = True

    return state





**Para habilitar o deshabilitar los logs:**

Si deseas desactivar los logs, puedes hacerlo configurando la siguiente variable:

```bash
DISABLE_LOGGING=true
```

De lo contrario, si prefieres mantener los logs activos, asegúrate de que esté configurado como:

```bash
DISABLE_LOGGING=false
```



In [5]:
graph = StateGraph(AgentState)

# 🧠 Nodes
graph.add_node("classify", classify_tasks)
graph.add_node("task_weather", get_weather)
graph.add_node("task_exchange", get_exchange_rate)
graph.add_node("task_news", get_news)
graph.add_node("task_order", order_tasks)
graph.add_node("handle_error", error_handler)
graph.add_node("check_completion", check_completion)
graph.add_node("aggregate", aggregator)

# ▶️ Start → classify
graph.add_edge(START, "classify")

# 🔀 Conditional route from classify
def route_tasks(state):
    tasks = []
    if state["tasks_to_do"].get("weather", False):
        tasks.append("task_weather")
    if state["tasks_to_do"].get("exchange", False):
        tasks.append("task_exchange")
    if state["tasks_to_do"].get("news", False):
        tasks.append("task_news")
    tasks.append("task_order")  # always
    print("tasks", tasks)
    return tasks

graph.add_conditional_edges("classify", route_tasks)

def determine_next(state, task_name):
    errors = state.get("error", {})
    key = task_to_error_key.get(task_name)
    if errors.get(key):
        print(f"⚠️ Task failed: {task_name} → {errors[key]}")
        return ["handle_error"]
    return ["check_completion"]

# Each task → error or completion
for task in ["task_weather", "task_exchange", "task_news", "task_order"]:
    graph.add_conditional_edges(
        task,
        lambda state, task_name=task: determine_next(state, task_name)
    )

# 🛠️ Handle error → Check completion
graph.add_edge("handle_error", "check_completion")

# ✅ Check if all done → Aggregate
graph.add_conditional_edges(
    "check_completion",
    lambda state: ["aggregate"] if state.get("ready_to_aggregate", False) else []
)

# 🔚 Aggregate → END
graph.add_edge("aggregate", END)


# 🔄 Determine next step: error or completion
task_to_error_key = {
    "task_weather": "weather",
    "task_exchange": "exchange",
    "task_news": "news",
    "task_order": "order"
}


# 🚀 Compile app
app = graph.compile()


In [6]:
# print(app.get_graph().draw_mermaid())

# Test de Agente 

In [7]:
from typing import List, Dict, Any, Optional
from langchain.schema import HumanMessage, BaseMessage

# Ejecución de prueba
inputs = {
    "messages": [HumanMessage(content="Quiero saber el clima en new york y como esta el dolar con respecto al peso mexicano y las ultimas noticias")],
    "order_task": None,
    "task_completed": {},
    "results": {},
    "error": {},
    "tasks_to_do": {},
    "ready_to_aggregate": False,
}

response = app.invoke(inputs)


[2025-04-25 04:24:33,359][WeatherAgent][INFO] Mensaje recibido: Quiero saber el clima en new york y como esta el dolar con respecto al peso mexicano y las ultimas noticias
[2025-04-25 04:24:34,030][WeatherAgent][INFO] Tareas clasificadas: {'weather': True, 'exchange': True, 'news': True}
[2025-04-25 04:24:34,032][WeatherAgent][INFO] Processing user message: Quiero saber el clima en new york y como esta el dolar con respecto al peso mexicano y las ultimas noticias
[2025-04-25 04:24:34,035][WeatherAgent][INFO] Processing user message: Quiero saber el clima en new york y como esta el dolar con respecto al peso mexicano y las ultimas noticias
[2025-04-25 04:24:34,036][WeatherAgent][INFO] Tareas detectadas: []


tasks ['task_weather', 'task_exchange', 'task_news', 'task_order']


[2025-04-25 04:24:34,379][WeatherAgent][INFO] City extracted: 'New York'
[2025-04-25 04:24:34,381][WeatherAgent][INFO] Fetching weather for: New York
[2025-04-25 04:24:34,408][WeatherAgent][INFO] Detected country code: us
[2025-04-25 04:24:34,421][WeatherAgent][INFO] Successfully extracted currency codes: ['USD', 'MXN']
[2025-04-25 04:24:34,422][WeatherAgent][INFO] Detected currencies: USD -> MXN
[2025-04-25 04:24:34,550][WeatherAgent][INFO] Found headlines: Headlines in US: LIVE UPDATES: Thousands bid farewell to Pope Francis as Vatican looks to extend viewing hours - Catholic News Agency, Everything You Need to Know About Preordering a Nintendo Switch 2 - WIRED, 17 Modifiable Lifestyle Factors That Protect Your Brain From Aging and Dementia - SciTechDaily
[2025-04-25 04:24:34,553][WeatherAgent][INFO] Orden propuesto por LLM: {'weather': 1, 'exchange': 2, 'noticias': 3}
[2025-04-25 04:24:34,565][WeatherAgent][INFO] Exchange rate obtained: 1 USD = 19.5951 MXN
[2025-04-25 04:24:35,256][

Tareas a verificar: ['exchange', 'news', 'order', 'weather']
{'messages': [HumanMessage(content='Quiero saber el clima en new york y como esta el dolar con respecto al peso mexicano y las ultimas noticias', additional_kwargs={}, response_metadata={}, id='14ecb8a2-33e2-45f8-aa85-0d3b6c3e8d02')], 'order_task': {'weather': 1, 'exchange': 2, 'noticias': 3}, 'error': {}, 'results': {'exchange': ['1 USD = 19.5951 MXN'], 'news': ['Headlines in US: LIVE UPDATES: Thousands bid farewell to Pope Francis as Vatican looks to extend viewing hours - Catholic News Agency, Everything You Need to Know About Preordering a Nintendo Switch 2 - WIRED, 17 Modifiable Lifestyle Factors That Protect Your Brain From Aging and Dementia - SciTechDaily'], 'weather': ['The weather in New York is clear sky with a temperature of 18.92°C.']}, 'task_completed': {'exchange': True, 'news': True, 'order': True, 'weather': True}, 'tasks_to_do': {'weather': True, 'exchange': True, 'news': True}, 'ready_to_aggregate': False, 

[2025-04-25 04:24:36,014][WeatherAgent][INFO] Mensaje procesado para 'weather': ¡Hola! Aquí te dejo el pronóstico del clima para Nueva York: cielo despejado con una temperatura de 18.92°C. ¡Que tengas un excelente día! 🌞🌇
[2025-04-25 04:24:36,725][WeatherAgent][INFO] Mensaje procesado para 'exchange': ¡Hola! Aquí te dejo el tipo de cambio actual: 1 USD = 19.5951 MXN. ¡Espero que te sea útil! ¡Que tengas un excelente día!


In [8]:
for item in (response["results"]['aggregator']):
    print(item)

¡Hola! Aquí te dejo el pronóstico del clima para Nueva York: cielo despejado con una temperatura de 18.92°C. ¡Que tengas un excelente día! 🌞🌇
¡Hola! Aquí te dejo el tipo de cambio actual: 1 USD = 19.5951 MXN. ¡Espero que te sea útil! ¡Que tengas un excelente día!
